In [1]:
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen
from astrosite_dataset import build_merge_dataset
from spinnaker_loader import EventsLoader

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np
import itertools

from tonic.slicers import SliceByTime
from tonic import SlicedDataset, transforms

/home/jules/miniconda3/envs/satellite/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset_path = '../dataset/recordings'

train_target_list = ['50574', '47851', '37951', '39533', '43751', '32711', '27831', '45465',
       '46826', '42942', '42741', '41471', '43873', '40982', '41725', '43874']

test_target_list = [ '27711', '40892', '50005', '44637']

train_set = build_merge_dataset(dataset_path, split=train_target_list, metadata_paths=['metadata/train_event','metadata/train_label'])
test_set = build_merge_dataset(dataset_path, split=test_target_list, metadata_paths=['metadata/test_event','metadata/test_label'])

Metadata written to metadata/train_event/slice_metadata.h5.
Metadata written to metadata/train_label/slice_metadata.h5.
Metadata written to metadata/test_event/slice_metadata.h5.
Metadata written to metadata/test_label/slice_metadata.h5.


In [4]:
# dataloader arguments
batch_size = 128

train_kwargs = {
        'shuffle': True,
        'batch_size': batch_size,
        'num_workers': 2,
    }
test_kwargs = {
        'shuffle': True,
        'batch_size': batch_size,
        'num_workers': 2,
    }

print(len(train_set))
print(len(test_set))
train_loader = torch.utils.data.DataLoader(train_set, **train_kwargs)
test_loader = torch.utils.data.DataLoader(test_set, **test_kwargs)


dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
loss = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(net.parameters(), lr=5e-4, betas=(0.9, 0.999))

11693
3587


In [6]:
data, targets, object_id = train_set[0]
data = data.to(device)
targets = targets.to(device)

In [ ]:
from utils import train_printer, print_batch_accuracy

num_epochs = 1
num_steps = 8
loss_hist = []
test_loss_hist = []
counter = 0

# Outer training loop
for epoch in range(num_epochs):
    iter_counter = 0
    train_batch = iter(train_loader)

    # Minibatch training loop
    for data, targets in train_batch:
        data = data.to(device)
        targets = targets.to(device)

        # forward pass
        net.train()
        spk_rec, mem_rec = net(data.view(batch_size, -1))

        # initialize the loss & sum over time
        loss_val = torch.zeros((1), dtype=dtype, device=device)
        for step in range(num_steps):
            loss_val += loss(mem_rec[step], targets)

        # Gradient calculation + weight update
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

        # Store loss history for future plotting
        loss_hist.append(loss_val.item())

        # Test set
        with torch.no_grad():
            net.eval()
            test_data, test_targets = next(iter(test_loader))
            test_data = test_data.to(device)
            test_targets = test_targets.to(device)

            # Test set forward pass
            test_spk, test_mem = net(test_data.view(batch_size, -1))

            # Test set loss
            test_loss = torch.zeros((1), dtype=dtype, device=device)
            for step in range(num_steps):
                test_loss += loss(test_mem[step], test_targets)
            test_loss_hist.append(test_loss.item())

            # Print train/test loss/accuracy
            if counter % 50 == 0:
                train_printer(
                    data, targets, epoch,
                    counter, iter_counter,
                    loss_hist, test_loss_hist,
                    test_data, test_targets)
            counter += 1
            iter_counter +=1